In [8]:
!nvidia-smi

Thu Jun  3 08:05:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    36W / 250W |   1497MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import re
import random
from tqdm import tqdm
import tensorflow as tf
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.models import Sequential, Model, load_model

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd '/content/drive/Shareddrives/utshabkg/ML_Competition-AND-Practice/Competition/HackerEarth Fast, Furious and Insured/'

/content/drive/Shareddrives/utshabkg/ML_Competition-AND-Practice/Competition/HackerEarth Fast, Furious and Insured


In [ ]:
# densenet = pd.read_csv('Original_data/densenet_scores.csv')
# resnet = pd.read_csv('Original_data/resnet_scores.csv')
# vgg16 = pd.read_csv('Original_data/vgg16_scores.csv')
# vgg19 = pd.read_csv('Original_data/vgg19_scores.csv')

In [3]:
from IPython.display import display_html

In [ ]:
# with pd.option_context('display.precision', 3):
#     df1_styler = vgg16.style.set_table_attributes("style='display:inline'").set_caption('Vgg16')
#     df2_styler = vgg19.style.set_table_attributes("style='display:inline'").set_caption('Vgg19')
#     df3_styler = densenet.style.set_table_attributes("style='display:inline'").set_caption('Densenet')
#     df4_styler = resnet.style.set_table_attributes("style='display:inline'").set_caption('Resnet')

# display_html(df1_styler._repr_html_()+df2_styler._repr_html_()+df3_styler._repr_html_()+df4_styler._repr_html_(), raw=True)

In [6]:
model1 = load_model('densenet/densenet_all-0.987.hdf5')

In [9]:
!/usr/local/cuda/bin/nvcc --version
# Check if GPU is selected as hardware accelerator
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if not '/device:GPU:0' in device_name:
    print('\nERROR: GPU is not selected as hardware accelerator!')
else:
    print(device_name)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0
/device:GPU:0


In [10]:
!git clone https://github.com/AlexeyAB/darknet
%cd darknet
!ls
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!make

Cloning into 'darknet'...
remote: Enumerating objects: 15054, done.
remote: Total 15054 (delta 0), reused 0 (delta 0), pack-reused 15054
Receiving objects: 100% (15054/15054), 13.43 MiB | 5.18 MiB/s, done.
Resolving deltas: 100% (10234/10234), done.
Checking out files: 100% (2028/2028), done.
/content/drive/Shareddrives/utshabkg/ML_Competition-AND-Practice/Competition/HackerEarth Fast, Furious and Insured/darknet
3rdparty		darknet_video.py       net_cam_v4.sh
build			data		       README.md
build.ps1		image_yolov3.sh        results
cfg			image_yolov4.sh        scripts
cmake			include		       src
CMakeLists.txt		json_mjpeg_streams.sh  vcpkg.json
DarknetConfig.cmake.in	LICENSE		       video_yolov3.sh
darknet_images.py	Makefile	       video_yolov4.sh
darknet.py		net_cam_v3.sh
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/inclu

In [21]:
# Creating a single function which predicts wether Car is damaged or not and localizing the damage, severity of damage for a single img
def report(img_path):
    Condition = [] 
    report_pred = []
    #Creating Train and test txt files:
    test_file = open('/content/drive/Shareddrives/utshabkg/ML_Competition-AND-Practice/Competition/HackerEarth Fast, Furious and Insured/test.txt','w')
    test_file.write('/content/drive/Shareddrives/utshabkg/ML_Competition-AND-Practice/Competition/HackerEarth Fast, Furious and Insured/'+img_path+'\n')
    test_file.close()
    
    img = load_img('/content/drive/Shareddrives/utshabkg/ML_Competition-AND-Practice/Competition/HackerEarth Fast, Furious and Insured/'+img_path,target_size = (256,256))
    #Converting into array
    img_arr = img_to_array(img)
    img_arr = img_arr.reshape((1,) + img_arr.shape)
    
    #Checking if Damaged or not
    s1_pred = model1.predict(img_arr)
    if s1_pred <=0.5:
        report_pred.append('Damaged')
        print('The Car is Damaged ')
        return 1
    else:
        report_pred.append('Not Damaged')
        print('Are you sure the vehicle is Damaged? Please upload another image.')
        return 0
    # #Checking for Damage Localization
    # s2_pred = Stage_2_model.predict(img_arr)
    # n = np.argmax(s2_pred)
    # if n == 0:
    #     report_pred.append('Front')
    # elif n ==1:
    #     report_pred.append('Rear')
    # else:
    #     report_pred.append('Side')
    
    # #Checking for Damage Severity
    # s3_pred = Stage_3_model.predict(img_arr)
    # c = np.argmax(s3_pred)
    # if c == 0:
    #     report_pred.append('Minor')
    # elif c ==1:
    #     report_pred.append('Moderate')
    # elif c==2:
    #     report_pred.append('Severe')

    # image_path = '/content/drive/Shareddrives/utshabkg/ML_Competition-AND-Practice/Competition/HackerEarth Fast, Furious and Insured/'+img_path
        
    #Using YOLO to detect the damage type
    # temp = !./darknet detector test /content/YOLO/custom.data /content/YOLO/yolov3_custom.cfg /content/models_weights/yolov3_custom_4000.weights -dont_show </content/test.txt> result.txt

    #Final report
    # print('The Car is '+report_pred[0])
    # print('The Car is '+report_pred[0]+' on the '+report_pred[1]+' suffering '+report_pred[2]+' damages.')

    # %matplotlib inline
    # image = cv2.imread('predictions.jpg')
    # fig = plt.gcf()
    # fig.set_size_inches(18, 10)
    # plt.axis("off")
    # plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    # plt.show()

In [31]:
%cd '/content/drive/Shareddrives/utshabkg/ML_Competition-AND-Practice/Competition/HackerEarth Fast, Furious and Insured/'

/content/drive/Shareddrives/utshabkg/ML_Competition-AND-Practice/Competition/HackerEarth Fast, Furious and Insured


In [37]:
condition = []
for i in range(len(os.listdir('dataset/testImages/'))):
    ins = [os.listdir('dataset/testImages/')[i], report('dataset/testImages/'+os.listdir('dataset/testImages/')[i])]
    condition.append(ins)
    if i%100==0:
        print(f'{i} cars tested! ')

Are you sure the vehicle is Damaged. Please upload another image?
The Car is Damaged 
Are you sure the vehicle is Damaged. Please upload another image?
The Car is Damaged 
Are you sure the vehicle is Damaged. Please upload another image?
Are you sure the vehicle is Damaged. Please upload another image?
The Car is Damaged 
Are you sure the vehicle is Damaged. Please upload another image?
The Car is Damaged 
The Car is Damaged 
The Car is Damaged 
The Car is Damaged 
The Car is Damaged 
The Car is Damaged 
The Car is Damaged 
The Car is Damaged 
The Car is Damaged 
The Car is Damaged 
The Car is Damaged 
The Car is Damaged 
The Car is Damaged 
The Car is Damaged 
Are you sure the vehicle is Damaged. Please upload another image?
The Car is Damaged 
The Car is Damaged 
Are you sure the vehicle is Damaged. Please upload another image?
The Car is Damaged 
The Car is Damaged 
The Car is Damaged 
The Car is Damaged 
The Car is Damaged 
The Car is Damaged 
The Car is Damaged 
The Car is Damaged

In [42]:
len(condition)

600

In [46]:
test_result = pd.DataFrame(condition, columns = ['Image_path', 'Condition'])
test_result.to_csv('test_result.csv', index=False)
test_result

,Image_path,Condition
0,img_4511084.jpg,0
1,img_4511787.jpg,1
2,img_4512558.jpg,0
3,img_4513036.jpg,1
4,img_4513264.jpg,0
...,...,...
595,img_776870.jpg,0
596,img_7768744.jpg,1
597,img_7768772.jpg,1
598,img_7768900.jpg,1
